# Venture Funding with Deep Learning

In [99]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

## Prepare the data to be used on a neural network model

### Step 1: Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define your features and target variables. 

In [100]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv("../Desktop/csv/applicants_data.csv")

# Review the DataFrame
applicant_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [101]:
# Review the data types associated with the columns
applicant_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


### Step 2: Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.

In [102]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=["EIN", 'NAME'])

# Review the DataFrame
applicant_data_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [103]:
list(applicant_data_df.dtypes[applicant_data_df.dtypes == "object"].index)

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [104]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes == 'object'].index)



# Display the categorical variables list
categorical_variables

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [105]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [106]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])

In [107]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)

# Review the DataFrame
encoded_df

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

In [108]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = applicant_data_df.drop(columns = categorical_variables)

# Review the Dataframe
encoded_df

,STATUS,ASK_AMT,IS_SUCCESSFUL
0,1,5000,1
1,1,108590,1
2,1,5000,0
3,1,6692,1
4,1,142590,1
...,...,...,...
34294,1,5000,0
34295,1,5000,0
34296,1,5000,0
34297,1,5000,1


### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 

In [109]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df['IS_SUCCESSFUL']

# Display a sample of y
y

0        1
1        1
2        0
3        1
4        1
        ..
34294    0
34295    0
34296    0
34297    1
34298    0
Name: IS_SUCCESSFUL, Length: 34299, dtype: int64

In [110]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns='IS_SUCCESSFUL')

# Review the features DataFrame
X

,STATUS,ASK_AMT
0,1,5000
1,1,108590
2,1,5000
3,1,6692
4,1,142590
...,...,...
34294,1,5000
34295,1,5000
34296,1,5000
34297,1,5000


### Step 6: Split the features and target sets into training and testing datasets.

In [111]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [112]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled =  X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile and Evaluate a Binary Classification Model Using a Neural Network
### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

In [113]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

2

In [114]:
# Define the number of neurons in the output layer
number_output_neurons = 10

In [115]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 15

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

15

In [116]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  5

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

5

In [117]:
# Create the Sequential model instance
nn = Sequential()

In [118]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [119]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [120]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(1))

In [121]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 15)                45        
                                                                 
 dense_20 (Dense)            (None, 5)                 80        
                                                                 
 dense_21 (Dense)            (None, 1)                 6         
                                                                 
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.

In [122]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [123]:
# Fit the model using 50 epochs and the training data
nn.fit(X_train_scaled,y_train, 
                    epochs=20,
                    batch_size=100,
                    shuffle=True)

Epoch 1/20
258/258 [==============================] - 1s 2ms/step - loss: 1.2394 - accuracy: 0.4657
Epoch 2/20
258/258 [==============================] - 1s 2ms/step - loss: 0.7117 - accuracy: 0.5123
Epoch 3/20
258/258 [==============================] - 1s 2ms/step - loss: 0.7032 - accuracy: 0.5365
Epoch 4/20
258/258 [==============================] - 1s 3ms/step - loss: 0.6970 - accuracy: 0.5373
Epoch 5/20
258/258 [==============================] - 1s 3ms/step - loss: 0.6932 - accuracy: 0.5376
Epoch 6/20
258/258 [==============================] - 1s 3ms/step - loss: 0.6916 - accuracy: 0.5376
Epoch 7/20
258/258 [==============================] - 1s 2ms/step - loss: 0.6925 - accuracy: 0.5374
Epoch 8/20
258/258 [==============================] - 1s 3ms/step - loss: 0.6921 - accuracy: 0.5375
Epoch 9/20
258/258 [==============================] - 1s 2ms/step - loss: 0.6914 - accuracy: 0.5376
Epoch 10/20
258/258 [==============================] - 1s 2ms/step - loss: 0.6918 - accuracy: 0.5375

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.

In [124]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7077 - accuracy: 0.5333 - 339ms/epoch - 1ms/step
Loss: 0.7077332139015198, Accuracy: 0.5332944393157959


### Step 4: Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`. 

In [125]:
# Set the model's file path
file_path = Path("FINTECH/Week_13/AlphabetSoup.h5")

# Export your model to a HDF5 file
nn.save(file_path)

## Optimize the neural network model

### Alternative Model 1

In [126]:
# Define the the number of inputs (features) to the model
number_input_features_1 = len(X_train.iloc[0])

# Review the number of features
number_input_features_1

2

In [127]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 40

In [128]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = 100

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

100

In [129]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 =  50

# Review the number of hidden nodes in the second layer
hidden_nodes_layer2_A1

50

In [130]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [131]:
# First and Second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, activation="relu", input_dim=number_input_features))

nn_A1.add(Dense(units=hidden_nodes_layer2_A1, activation='relu', input_dim=number_input_features))

# Output layer
nn_A1.add(Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_A1.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 100)               300       
                                                                 
 dense_23 (Dense)            (None, 50)                5050      
                                                                 
 dense_24 (Dense)            (None, 1)                 51        
                                                                 
Total params: 5,401
Trainable params: 5,401
Non-trainable params: 0
_________________________________________________________________


In [132]:
# Compile the Sequential model
nn_A1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [133]:
# Fit the model using 50 epochs and the training data
nn_A1.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6929 - accuracy: 0.5345
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6919 - accuracy: 0.5345
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6920 - accuracy: 0.5334
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.6918 - accuracy: 0.5345
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6918 - accuracy: 0.5353
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6917 - accuracy: 0.5344
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5348
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6910 - accuracy: 0.5348
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.6910 - accuracy: 0.5349
Epoch 10/50
804/804 [==============================] - 2s 3ms/step - loss: 0.6908 - accuracy: 0.5353

#### Alternative Model 2

In [134]:
# Define the the number of inputs (features) to the model
number_input_features_2 = len(X_train.iloc[0])

# Review the number of features
number_input_features_2

2

In [135]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 20

In [136]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = 50

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

50

In [137]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 = 25

# Review the number of hidden nodes in the second layer
hidden_nodes_layer2_A2

25

In [138]:
# Define the number of hidden nodes for the third hidden layer
hidden_nodes_layer3_A2 = 10

# Review the number of hidden nodes in the third layer
hidden_nodes_layer3_A2

10

In [139]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [140]:
# First and Second and Third hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, activation="relu", input_dim=number_input_features_2))

nn_A2.add(Dense(units=hidden_nodes_layer2_A2, activation="relu", input_dim=number_input_features_2))

nn_A2.add(Dense(units=hidden_nodes_layer3_A2, activation="relu", input_dim=number_input_features_2))

# Output layer
nn_A2.add(Dense(units=1, activation="sigmoid")) 

# Check the structure of the model
nn_A2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 50)                150       
                                                                 
 dense_26 (Dense)            (None, 25)                1275      
                                                                 
 dense_27 (Dense)            (None, 10)                260       
                                                                 
 dense_28 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,696
Trainable params: 1,696
Non-trainable params: 0
_________________________________________________________________


In [141]:
# Compile the model
nn_A2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [142]:
# Fit the model
nn_A2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6919 - accuracy: 0.5334
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6910 - accuracy: 0.5342
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6909 - accuracy: 0.5346
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6913 - accuracy: 0.5352
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6908 - accuracy: 0.5361
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5345
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6907 - accuracy: 0.5356
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6908 - accuracy: 0.5353
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5355
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6907 - accura

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [143]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
268/268 - 0s - loss: 0.7077 - accuracy: 0.5333 - 335ms/epoch - 1ms/step
Loss: 0.7077332139015198, Accuracy: 0.5332944393157959


In [144]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
268/268 - 0s - loss: 0.6908 - accuracy: 0.5338 - 358ms/epoch - 1ms/step
Loss: 0.6908370852470398, Accuracy: 0.5337609052658081


In [145]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
268/268 - 0s - loss: 0.6912 - accuracy: 0.5336 - 356ms/epoch - 1ms/step
Loss: 0.6911563277244568, Accuracy: 0.5336443185806274


### Step 3: Save each of your alternative models as an HDF5 file.

In [146]:
# Set the file path for the first alternative model
file_path = Path("FINTECH/Week_13/nn_A1.h5")

# Export your model to a HDF5 file
nn_A1.save(file_path)

In [147]:
# Set the file path for the second alternative model
file_path = Path("FINTECH/Week_13/nn_A2.h5")

# Export your model to a HDF5 file
nn_A2.save(file_path)